# Приложение "Морской бой"

## Класс точки на доске
1. Хранит в себе x, y координаты точки
2. `__eq__` - магический метод, чтобы сравнивать точки на равенство
3. `__repr__` - магический метод, который вызывается при выводе объекта в консоль

In [1]:
class Dot:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __repr__(self):
        return f"({self.x}, {self.y})"


## Класс корабля на доске
1. Храним точку носа корабля
2. Храним длину корабля
3. Храним количество оставшихся целых частей корабля
4. Храним ориентацию корабля в пространстве
5. Метод (и `property`) `dots` возвращает список из точек, принадлежащих кораблю
6. Метод `shooten` проверяет, что выстрел попал в корабль

In [9]:
class Ship:
    def __init__(self, bow, l, o):
        self.bow = bow
        self.l = l
        self.o = o
        self.lives = l
    
    @property
    def dots(self):
        ship_dots = []
        for i in range(self.l):
            cur_x = self.bow.x 
            cur_y = self.bow.y
            
            if self.o == 0:
                cur_x += i
            
            elif self.o == 1:
                cur_y += i
            
            ship_dots.append(Dot(cur_x, cur_y))
        
        return ship_dots
    
    def shooten(self, shot):
        return shot in self.dots


## Классы исключений
1. `BoardException` - класс-родитель всех исключений
2. `BoardOutException` - исключения, когда пользователь стреляет за границы доски
3. `BoardUsedException` - исключения, когда пользователь стреляет в точку, куда уже стрелял
4. `BoardWrongShipException` - исключения, когда пытаемся поставить корабль на доску в недопустимое место

In [11]:
class BoardException(Exception):
    pass

class BoardOutException(BoardException):
    def __str__(self):
        return "Вы пытаетесь выстрелить за доску!"

class BoardUsedException(BoardException):
    def __str__(self):
        return "Вы уже стреляли в эту клетку"

class BoardWrongShipException(BoardException):
    pass

## Класс игровой доски
1. `size` - размер доски
2. `hid` - скрыты ли корабли на доске
3. `field` - двумерный список с игровым полем
4. `busy` - список занятых точек
5. `ships` - список кораблей на доске
6. `__str__` - метод, который вызывается при печати доски
7. `add_ship` - метод, который добавляет корабль на доску
8. `contour` - метод, который обводит корабль по контуру
9. `out` - метод, который проверяет, вышла ли точка за границу доски
10. `shot` - метод, который совершает выстрел в точку. Возвращает `bool` значение - может ли игрок повторить выстрел
11. `begin` - метод, который вызывается в начале игры, после расстановки кораблей

In [12]:
class Board:
    def __init__(self, hid = False, size = 6):
        self.size = size
        self.hid = hid
        
        self.count = 0
        
        self.field = [ ["O"]*size for _ in range(size) ]
        
        self.busy = []
        self.ships = []
    
    def add_ship(self, ship):
        
        for d in ship.dots:
            if self.out(d) or d in self.busy:
                raise BoardWrongShipException()
        for d in ship.dots:
            self.field[d.x][d.y] = "■"
            self.busy.append(d)
        
        self.ships.append(ship)
        self.contour(ship)
            
    def contour(self, ship, verb = False):
        near = [
            (-1, -1), (-1, 0) , (-1, 1),
            (0, -1), (0, 0) , (0 , 1),
            (1, -1), (1, 0) , (1, 1)
        ]
        for d in ship.dots:
            for dx, dy in near:
                cur = Dot(d.x + dx, d.y + dy)
                if not(self.out(cur)) and cur not in self.busy:
                    if verb:
                        self.field[cur.x][cur.y] = "."
                    self.busy.append(cur)
    
    def __str__(self):
        res = ""
        res += "  | 1 | 2 | 3 | 4 | 5 | 6 |"
        for i, row in enumerate(self.field):
            res += f"\n{i+1} | " + " | ".join(row) + " |"
        
        if self.hid:
            res = res.replace("■", "O")
        return res
    
    def out(self, d):
        return not((0<= d.x < self.size) and (0<= d.y < self.size))

    def shot(self, d):
        if self.out(d):
            raise BoardOutException()
        
        if d in self.busy:
            raise BoardUsedException()
        
        self.busy.append(d)
        
        for ship in self.ships:
            if d in ship.dots:
                ship.lives -= 1
                self.field[d.x][d.y] = "X"
                if ship.lives == 0:
                    self.count += 1
                    self.contour(ship, verb = True)
                    print("Корабль уничтожен!")
                    return False
                else:
                    print("Корабль ранен!")
                    return True
        
        self.field[d.x][d.y] = "."
        print("Мимо!")
        return False
    
    def begin(self):
        self.busy = []


## Классы игроков
1. `Player` - клас - родитель всех игроков
2. `board` - доска игрока
3. `enemy` - вражескя доска
4. `move` - метод, который совершает ход игрока
5. `ask` - метод, который спрашивает у игрока ход
6. `AI` - класс компьютера
7. `AI.ask` - переопределили метод, который спрашивает ход, тут метод просто генерирует случайные числа
8. `User` - класс пользователя
9. `User.ask` - переопределили метод, чтобы он српашвал числа из консоли

In [13]:
class Player:
    def __init__(self, board, enemy):
        self.board = board
        self.enemy = enemy
    
    def ask(self):
        raise NotImplementedError()
    
    def move(self):
        while True:
            try:
                target = self.ask()
                repeat = self.enemy.shot(target)
                return repeat
            except BoardException as e:
                print(e)

class AI(Player):
    def ask(self):
        d = Dot(randint(0,5), randint(0, 5))
        print(f"Ход компьютера: {d.x+1} {d.y+1}")
        return d

class User(Player):
    def ask(self):
        while True:
            cords = input("Ваш ход: ").split()
            
            if len(cords) != 2:
                print(" Введите 2 координаты! ")
                continue
            
            x, y = cords
            
            if not(x.isdigit()) or not(y.isdigit()):
                print(" Введите числа! ")
                continue
            
            x, y = int(x), int(y)
            
            return Dot(x-1, y-1)


## Основной класс игры
1. `random_place` - метод, который создаёт случайную расстановку кораблей на доске, если расстановка не удалась, возвращает `None`
2. `random_board` - метод, который гарантированно генерирует случайную доску с кораблями
3. `pl' - доска пользователя
4. `co` - доска компьютера
5. `ai` - игрок-компьютер
6. `us` - игрок-пользователь
7. `greet` - метод, печатающий приветствие
8. `loop` - метод с игровым циклом
9. `start` - метод с началом игры

In [17]:
class Game:
    def __init__(self, size = 6):
        self.size = size
        pl = self.random_board()
        co = self.random_board()
        co.hid = True
        
        self.ai = AI(co, pl)
        self.us = User(pl, co)
    
    def random_board(self):
        board = None
        while board is None:
            board = self.random_place()
        return board
    
    def random_place(self):
        lens = [3, 2, 2, 1, 1, 1, 1]
        board = Board(size = self.size)
        attempts = 0
        for l in lens:
            while True:
                attempts += 1
                if attempts > 2000:
                    return None
                ship = Ship(Dot(randint(0, self.size), randint(0, self.size)), l, randint(0,1))
                try:
                    board.add_ship(ship)
                    break
                except BoardWrongShipException:
                    pass
        board.begin()
        return board

    def greet(self):
        print("-------------------")
        print("  Приветсвуем вас  ")
        print("      в игре       ")
        print("    морской бой    ")
        print("-------------------")
        print(" формат ввода: x y ")
        print(" x - номер строки  ")
        print(" y - номер столбца ")
    
    
    def loop(self):
        num = 0
        while True:
            print("-"*20)
            print("Доска пользователя:")
            print(self.us.board)
            print("-"*20)
            print("Доска компьютера:")
            print(self.ai.board)
            if num % 2 == 0:
                print("-"*20)
                print("Ходит пользователь!")
                repeat = self.us.move()
            else:
                print("-"*20)
                print("Ходит компьютер!")
                repeat = self.ai.move()
            if repeat:
                num -= 1
            
            if self.ai.board.count == 7:
                print("-"*20)
                print("Пользователь выиграл!")
                break
            
            if self.us.board.count == 7:
                print("-"*20)
                print("Компьютер выиграл!")
                break
            num += 1
            
    def start(self):
        self.greet()
        self.loop()


## Запуск игры

In [18]:
g = Game()
g.start()

■ | O | O | O | O |
3 | O | O | O | O | . | . |
4 | O | O | ■ | ■ | X | O |
5 | ■ | O | O | O | O | O |
6 | ■ | O | O | ■ | O | ■ |
--------------------
Доска компьютера:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | . | . | O | O | O | O |
2 | O | O | O | O | O | O |
3 | O | O | O | O | O | O |
4 | O | O | O | O | O | O |
5 | O | O | O | O | O | O |
6 | O | O | O | O | O | O |
--------------------
Ходит пользователь!
Мимо!
--------------------
Доска пользователя:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | O | ■ | O | ■ | O | ■ |
2 | O | ■ | O | O | O | O |
3 | O | O | O | O | . | . |
4 | O | O | ■ | ■ | X | O |
5 | ■ | O | O | O | O | O |
6 | ■ | O | O | ■ | O | ■ |
--------------------
Доска компьютера:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | . | . | . | O | O | O |
2 | O | O | O | O | O | O |
3 | O | O | O | O | O | O |
4 | O | O | O | O | O | O |
5 | O | O | O | O | O | O |
6 | O | O | O | O | O | O |
--------------------
Ходит компьютер!
Ход компьютера: 1 6
Корабль уничтожен!
--------------------
Доска пользовате